In [ ]:
from pathlib import Path
from registry import RegistryBase
from helpers import REGISTRY_LOCAL_IP
import numpy as np

registry = RegistryBase(base=REGISTRY_LOCAL_IP)

# NO MOTION IMAGE

lst = [
    Path('verdis/gadstrup/5/20260112_143000/20260112_143000_833817.jpg'), 
    Path('verdis/gadstrup/5/20260112_143000/20260112_143004_384804.jpg'), 
    Path('verdis/gadstrup/5/20260112_143000/20260112_143007_395239.jpg')]

img1_no_motion = np.array(registry.GET(lst[0], registry.DefaultMarkers.ORIGINAL_MARKER))
img3_no_motion = np.array(registry.GET(lst[2], registry.DefaultMarkers.ORIGINAL_MARKER))

# MOTION IMAGE

lst = [
    Path('verdis/gadstrup/5/20260113_075023/20260113_075023_143102.jpg'), 
    Path('verdis/gadstrup/5/20260113_075023/20260113_075026_141045.jpg'), 
    Path('verdis/gadstrup/5/20260113_075023/20260113_075029_799467.jpg')]

img1_motion = np.array(registry.GET(lst[0], registry.DefaultMarkers.ORIGINAL_MARKER))
img3_motion = np.array(registry.GET(lst[2], registry.DefaultMarkers.ORIGINAL_MARKER))


: 

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

def raw_diff(img1, img2):
    """Naive absdiff — shows JPEG artifact noise everywhere."""
    g1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    g2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    return cv2.absdiff(g1, g2)

def threshold_diff(img1, img2, thresh=25):
    """Absdiff + binary threshold to kill low-level JPEG noise."""
    diff = raw_diff(img1, img2)
    _, mask = cv2.threshold(diff, thresh, 255, cv2.THRESH_BINARY)
    return mask

def blur_then_diff(img1, img2, ksize=5, thresh=20):
    """Blur both images first to smooth JPEG artifacts, then diff + threshold."""
    g1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    g2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    g1 = cv2.GaussianBlur(g1, (ksize, ksize), 0)
    g2 = cv2.GaussianBlur(g2, (ksize, ksize), 0)
    diff = cv2.absdiff(g1, g2)
    _, mask = cv2.threshold(diff, thresh, 255, cv2.THRESH_BINARY)
    return mask

def blur_diff_morph(img1, img2, ksize=5, thresh=20, morph_size=5):
    """Blur + diff + threshold + morphological open (erode then dilate) to clean up."""
    mask = blur_then_diff(img1, img2, ksize, thresh)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_size, morph_size))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    mask = cv2.dilate(mask, kernel, iterations=3)
    return mask

def median_blur_diff(img1, img2, ksize=5, thresh=20):
    """Median blur (better at removing salt-and-pepper JPEG noise) + diff."""
    g1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    g2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    g1 = cv2.medianBlur(g1, ksize)
    g2 = cv2.medianBlur(g2, ksize)
    diff = cv2.absdiff(g1, g2)
    _, mask = cv2.threshold(diff, thresh, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=2)
    return mask

def structural_similarity_diff(img1, img2, win_size=7):
    """Use SSIM — compares structure not pixels, very robust to compression artifacts."""
    from skimage.metrics import structural_similarity as ssim
    g1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    g2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    _, diff_map = ssim(g1, g2, full=True, win_size=win_size)
    # diff_map is 1.0 where identical, 0.0 where different
    diff_map = (1.0 - diff_map)  # invert so motion = bright
    diff_map = (diff_map * 255).astype(np.uint8)
    _, mask = cv2.threshold(diff_map, 30, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    return mask

def color_channel_diff(img1, img2, thresh=25):
    """Diff each color channel separately, combine with OR — catches color-only changes."""
    diff = cv2.absdiff(img1, img2)
    masks = []
    for c in range(3):
        _, m = cv2.threshold(diff[:, :, c], thresh, 255, cv2.THRESH_BINARY)
        masks.append(m)
    combined = masks[0] | masks[1] | masks[2]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    combined = cv2.morphologyEx(combined, cv2.MORPH_OPEN, kernel, iterations=2)
    combined = cv2.dilate(combined, kernel, iterations=2)
    return combined

# ---- Define all methods ----
methods = [
    ("1. Raw absdiff",              raw_diff),
    ("2. Threshold (t=25)",         threshold_diff),
    ("3. Gaussian blur + thresh",   blur_then_diff),
    ("4. Blur + thresh + morph",    blur_diff_morph),
    ("5. Median blur + thresh",     median_blur_diff),
    ("6. SSIM diff",                structural_similarity_diff),
    ("7. Color channel diff",       color_channel_diff),
]

# ---- Plot side by side: No Motion (left) vs Motion (right) ----
fig, axes = plt.subplots(len(methods), 2, figsize=(14, 4 * len(methods)))
fig.suptitle("JPEG-Robust Motion Detection Methods\nLeft = No Motion | Right = Motion", 
             fontsize=16, fontweight='bold', y=1.01)

for i, (name, fn) in enumerate(methods):
    result_no = fn(img1_no_motion, img3_no_motion)
    result_mo = fn(img1_motion, img3_motion)
    
    axes[i, 0].imshow(result_no, cmap='gray', vmin=0, vmax=255)
    axes[i, 0].set_title(f"{name}\n(no motion)", fontsize=10)
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(result_mo, cmap='gray', vmin=0, vmax=255)
    axes[i, 1].set_title(f"{name}\n(motion)", fontsize=10)
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()